<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [27]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK',
       'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART',
       'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND',
       'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NAM-INDIA',
       'NATIONALUM', 'NAUKRI', 'PFIZER', 'PIDILITIND', 'RELIANCE', 'SAIL',
       'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS',
       'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'VOLTAS', 'WHIRLPOOL', 'WIPRO', 'ZEEL'], dtype=object)

In [28]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    57
1    12
Name: InPortfolio, dtype: int64

In [29]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    8
1    4
Name: CCIFlag, dtype: int64

In [30]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [31]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [32]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [33]:
# nifty in last 5 days
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-05 00:00:00+05:30,17992.0,^NSEI,4.28,0.45,0.27,18812.0,15294.0,76.70,23.30,41.0,5.0
2023-01-06 00:00:00+05:30,17859.0,^NSEI,3.49,0.42,0.23,18812.0,15294.0,72.92,27.08,38.0,5.0
2023-01-09 00:00:00+05:30,18101.0,^NSEI,4.86,0.38,0.19,18812.0,15294.0,79.80,20.20,47.0,4.0
2023-01-10 00:00:00+05:30,17914.0,^NSEI,3.76,0.34,0.11,18812.0,15294.0,74.48,25.52,42.0,5.0
2023-01-11 00:00:00+05:30,17896.0,^NSEI,3.64,0.31,0.04,18812.0,15294.0,73.95,26.05,41.0,5.0


In [34]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [35]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg']
df_prec_dev = df_prec_dev[cols]

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 12


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,Profit%FC,Profit%52W,Profit%Avg
21,VOLTAS.NS,831.0,-14.16,-2.27,-0.60,1315.0,778.0,9.86,90.14,57.0,974.0,821.0,43.4,156.00,17.0,58.0,38.0,19.0,60.0,40.0
1,BERGEPAINT.NS,561.0,-10.34,-0.96,-0.33,785.0,558.0,1.45,98.55,28.0,782.0,570.0,72.9,57.60,39.0,40.0,40.0,37.0,38.0,38.0
41,TECHM.NS,1006.0,-6.91,-2.04,-0.48,1684.0,935.0,9.53,90.47,45.0,1131.0,1048.0,18.7,18.50,12.0,67.0,40.0,8.0,61.0,34.0
20,TATAMOTORS.NS,418.0,-2.06,-0.23,-0.24,525.0,372.0,30.20,69.80,59.0,491.0,392.0,19.2,1000.00,17.0,26.0,22.0,25.0,34.0,30.0
2,ICICIPRULI.NS,462.0,-10.52,0.00,-0.85,615.0,440.0,12.40,87.60,49.0,636.0,484.0,60.9,77.70,38.0,33.0,36.0,31.0,27.0,29.0
5,MOTILALOFS.NS,712.0,-6.85,-0.92,0.29,960.0,654.0,18.84,81.16,58.0,930.0,733.0,19.7,9.76,31.0,35.0,33.0,27.0,31.0,29.0
26,MUTHOOTFIN.NS,1068.0,-2.83,-1.18,0.18,1502.0,956.0,20.49,79.51,46.0,1217.0,1082.0,12.5,11.40,14.0,41.0,28.0,12.0,39.0,26.0
7,NAUKRI.NS,3656.0,-9.29,-0.97,-0.50,5667.0,3363.0,12.70,87.30,30.0,4677.0,4154.0,78.8,15.90,28.0,55.0,42.0,13.0,36.0,24.0
10,BATAINDIA.NS,1637.0,-8.17,-0.56,-1.07,2020.0,1587.0,11.64,88.36,44.0,2042.0,1632.0,62.3,68.00,25.0,23.0,24.0,25.0,24.0,24.0
31,HDFCLIFE.NS,600.0,7.50,0.54,1.23,672.0,502.0,57.56,42.44,61.0,684.0,578.0,89.3,87.60,14.0,12.0,13.0,18.0,16.0,17.0


In [36]:
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg']
df_prec_dev = df_prec_dev[cols]

In [37]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 6


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
6,JUBLFOOD.NS,491.0,-11.65,-0.18,-2.04,785.0,463.0,8.76,91.24,31.0,636.0,87.9,68.4,30.0,60.0,45.0
45,WIPRO.NS,394.0,-9.25,-2.30,0.00,687.0,375.0,6.04,93.96,52.0,425.0,19.0,19.2,8.0,74.0,41.0
1,BERGEPAINT.NS,561.0,-10.34,-0.96,-0.33,785.0,558.0,1.45,98.55,28.0,782.0,72.9,57.6,39.0,40.0,40.0
41,TECHM.NS,1006.0,-6.91,-2.04,-0.48,1684.0,935.0,9.53,90.47,45.0,1131.0,18.7,18.5,12.0,67.0,40.0
42,LTIM.NS,4226.0,-8.50,-1.95,-1.04,7121.0,3796.0,12.92,87.08,39.0,4675.0,26.6,48.8,11.0,69.0,40.0
19,INFY.NS,1471.0,-3.02,-1.12,-0.39,1898.0,1351.0,21.98,78.02,39.0,1733.0,23.0,26.9,18.0,29.0,24.0


In [38]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 14


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
15,IBREALEST.NS,77.0,-2.72,-2.53,0.00,174.0,60.0,14.78,85.22,41.0,95.0,11.3,1000.00,23.0,126.0,74.0
6,JUBLFOOD.NS,491.0,-11.65,-0.18,-2.04,785.0,463.0,8.76,91.24,31.0,636.0,87.9,68.40,30.0,60.0,45.0
0,GODREJIND.NS,445.0,-1.71,-0.44,0.23,644.0,405.0,16.84,83.16,51.0,638.0,39.5,19.40,43.0,45.0,44.0
45,WIPRO.NS,394.0,-9.25,-2.30,0.00,687.0,375.0,6.04,93.96,52.0,425.0,19.0,19.20,8.0,74.0,41.0
42,LTIM.NS,4226.0,-8.50,-1.95,-1.04,7121.0,3796.0,12.92,87.08,39.0,4675.0,26.6,48.80,11.0,69.0,40.0
4,ZEEL.NS,229.0,-8.50,-0.40,-1.19,322.0,206.0,19.61,80.39,28.0,303.0,24.6,27.10,32.0,41.0,36.0
3,BAJFINANCE.NS,5896.0,-11.40,-0.39,-1.68,7824.0,5265.0,24.65,75.35,24.0,7966.0,50.8,36.00,35.0,33.0,34.0
8,NAM-INDIA.NS,251.0,-9.75,-1.08,-0.38,355.0,245.0,5.36,94.64,40.0,322.0,29.9,23.40,28.0,41.0,34.0
50,NATIONALUM.NS,85.0,2.05,-1.20,1.30,128.0,67.0,29.02,70.98,65.0,90.0,10.3,6.13,6.0,51.0,28.0
12,WHIRLPOOL.NS,1483.0,-8.02,-0.31,-0.46,1871.0,1413.0,15.21,84.79,41.0,1854.0,72.2,72.60,25.0,26.0,26.0


In [39]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 87]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
8,NAM-INDIA.NS,251.0,-9.75,-1.08,-0.38,355.0,245.0,5.36,94.64,40.0,322.0,29.9,23.4,28.0,41.0,34.0
45,WIPRO.NS,394.0,-9.25,-2.30,0.00,687.0,375.0,6.04,93.96,52.0,425.0,19.0,19.2,8.0,74.0,41.0
6,JUBLFOOD.NS,491.0,-11.65,-0.18,-2.04,785.0,463.0,8.76,91.24,31.0,636.0,87.9,68.4,30.0,60.0,45.0
33,TATAPOWER.NS,206.0,-9.32,-0.44,-0.91,287.0,195.0,11.79,88.21,40.0,232.0,20.4,25.9,13.0,39.0,26.0
42,LTIM.NS,4226.0,-8.50,-1.95,-1.04,7121.0,3796.0,12.92,87.08,39.0,4675.0,26.6,48.8,11.0,69.0,40.0


In [40]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] < 10]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
62,TATASTEEL.NS,119.0,33.54,3.37,1.82,119.0,44.0,99.80,0.20,63.0,118.0,6.0,5.05,-1.0,0.0,-0.0
40,AXISBANK.NS,950.0,22.39,1.55,0.55,962.0,625.0,96.36,3.64,58.0,1060.0,31.3,15.90,12.0,1.0,6.0
39,M&M.NS,1313.0,14.84,2.27,-0.24,1354.0,684.0,93.82,6.18,61.0,1475.0,20.5,19.50,12.0,3.0,8.0
64,INDIANB.NS,288.0,43.95,3.50,1.43,300.0,127.0,93.01,6.99,51.0,284.0,7.4,8.24,-1.0,4.0,2.0
37,FEDERALBNK.NS,135.0,21.31,1.80,0.74,139.0,82.0,92.37,7.63,50.0,152.0,11.0,11.40,13.0,3.0,8.0
57,SUNPHARMA.NS,1028.0,11.14,0.54,0.00,1052.0,791.0,90.80,9.20,60.0,1056.0,NaN,31.10,3.0,2.0,2.0
65,LT.NS,2125.0,16.33,0.99,0.58,2195.0,1453.0,90.61,9.39,55.0,2036.0,NaN,31.10,-4.0,3.0,-0.0


In [41]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
0,GODREJIND.NS,445.0,-1.71,-0.44,0.23,644.0,405.0,16.84,83.16,51.0,638.0,39.5,19.4,43.0,45.0,44.0
59,MCDOWELL-N.NS,868.0,3.27,-0.12,-0.22,947.0,732.0,63.49,36.51,42.0,885.0,74.5,56.4,2.0,9.0,6.0
12,WHIRLPOOL.NS,1483.0,-8.02,-0.31,-0.46,1871.0,1413.0,15.21,84.79,41.0,1854.0,72.2,72.6,25.0,26.0,26.0
16,TITAN.NS,2437.0,0.18,-0.12,-0.89,2770.0,1929.0,60.45,39.55,35.0,2980.0,78.0,68.4,22.0,14.0,18.0
33,TATAPOWER.NS,206.0,-9.32,-0.44,-0.91,287.0,195.0,11.79,88.21,40.0,232.0,20.4,25.9,13.0,39.0,26.0
43,DMART.NS,3873.0,-3.90,-0.20,-1.02,4577.0,3231.0,47.69,52.31,41.0,4282.0,138.7,109.0,11.0,18.0,14.0
4,ZEEL.NS,229.0,-8.50,-0.40,-1.19,322.0,206.0,19.61,80.39,28.0,303.0,24.6,27.1,32.0,41.0,36.0
17,BAJAJFINSV.NS,1400.0,-7.49,-0.33,-1.49,1836.0,1093.0,41.28,58.72,26.0,1691.0,39.2,40.8,21.0,31.0,26.0
3,BAJFINANCE.NS,5896.0,-11.40,-0.39,-1.68,7824.0,5265.0,24.65,75.35,24.0,7966.0,50.8,36.0,35.0,33.0,34.0
6,JUBLFOOD.NS,491.0,-11.65,-0.18,-2.04,785.0,463.0,8.76,91.24,31.0,636.0,87.9,68.4,30.0,60.0,45.0


In [42]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
50,NATIONALUM.NS,85.0,2.05,-1.20,1.30,128.0,67.0,29.02,70.98,65.0,90.0,10.3,6.13,6.0,51.0,28.0
0,GODREJIND.NS,445.0,-1.71,-0.44,0.23,644.0,405.0,16.84,83.16,51.0,638.0,39.5,19.40,43.0,45.0,44.0
52,HINDALCO.NS,491.0,14.44,-1.17,1.55,626.0,314.0,56.71,43.29,62.0,514.0,10.4,7.96,5.0,27.0,16.0
51,HCLTECH.NS,1055.0,5.06,-0.50,0.09,1291.0,865.0,44.55,55.45,51.0,1117.0,16.3,20.70,6.0,22.0,14.0
58,TCS.NS,3329.0,1.36,-0.52,0.33,3965.0,2974.0,35.79,64.21,54.0,3385.0,28.1,29.90,2.0,19.0,10.0


In [43]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
56,INDIGO.NS,2107.0,13.60,0.54,1.40,2265.0,1549.0,77.98,22.02,64.0,2176.0,30.7,1000.0,3.0,7.0,5.0
29,HDFCBANK.NS,1591.0,9.04,0.62,0.75,1662.0,1281.0,81.34,18.66,45.0,1813.0,26.0,21.2,14.0,4.0,9.0


In [44]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 9


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
52,HINDALCO.NS,491.0,14.44,-1.17,1.55,626.0,314.0,56.71,43.29,62.0,514.0,10.4,7.96,5.0,27.0,16.0
50,NATIONALUM.NS,85.0,2.05,-1.20,1.30,128.0,67.0,29.02,70.98,65.0,90.0,10.3,6.13,6.0,51.0,28.0
58,TCS.NS,3329.0,1.36,-0.52,0.33,3965.0,2974.0,35.79,64.21,54.0,3385.0,28.1,29.90,2.0,19.0,10.0
0,GODREJIND.NS,445.0,-1.71,-0.44,0.23,644.0,405.0,16.84,83.16,51.0,638.0,39.5,19.40,43.0,45.0,44.0
51,HCLTECH.NS,1055.0,5.06,-0.50,0.09,1291.0,865.0,44.55,55.45,51.0,1117.0,16.3,20.70,6.0,22.0,14.0
15,IBREALEST.NS,77.0,-2.72,-2.53,0.00,174.0,60.0,14.78,85.22,41.0,95.0,11.3,1000.00,23.0,126.0,74.0
45,WIPRO.NS,394.0,-9.25,-2.30,0.00,687.0,375.0,6.04,93.96,52.0,425.0,19.0,19.20,8.0,74.0,41.0
8,NAM-INDIA.NS,251.0,-9.75,-1.08,-0.38,355.0,245.0,5.36,94.64,40.0,322.0,29.9,23.40,28.0,41.0,34.0
42,LTIM.NS,4226.0,-8.50,-1.95,-1.04,7121.0,3796.0,12.92,87.08,39.0,4675.0,26.6,48.80,11.0,69.0,40.0


In [45]:
# star stocks performance

star_stocks = ['HINDUNILVR.NS', 'BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS',
          'CANBK.NS', 'TCS.NS', 'M&M.NS', 'LTIM.NS', 'ICICIBANK.NS',
          'AXISBANK.NS', 'HCLTECH.NS', 'TECHM.NS', 'WIPRO.NS',
          'INDUSINDBK.NS', 'JUBLFOOD.NS', 'DMART.NS', 'DABUR.NS',
          'TATAPOWER.NS', 'BHARTIARTL.NS']

df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)].sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
6,JUBLFOOD.NS,491.0,-11.65,-0.18,-2.04,785.0,463.0,8.76,91.24,31.0,636.0,87.9,68.40,30.0,60.0,45.0
45,WIPRO.NS,394.0,-9.25,-2.30,0.00,687.0,375.0,6.04,93.96,52.0,425.0,19.0,19.20,8.0,74.0,41.0
41,TECHM.NS,1006.0,-6.91,-2.04,-0.48,1684.0,935.0,9.53,90.47,45.0,1131.0,18.7,18.50,12.0,67.0,40.0
42,LTIM.NS,4226.0,-8.50,-1.95,-1.04,7121.0,3796.0,12.92,87.08,39.0,4675.0,26.6,48.80,11.0,69.0,40.0
3,BAJFINANCE.NS,5896.0,-11.40,-0.39,-1.68,7824.0,5265.0,24.65,75.35,24.0,7966.0,50.8,36.00,35.0,33.0,34.0
33,TATAPOWER.NS,206.0,-9.32,-0.44,-0.91,287.0,195.0,11.79,88.21,40.0,232.0,20.4,25.90,13.0,39.0,26.0
11,EICHERMOT.NS,3137.0,1.14,1.45,-1.63,3850.0,2241.0,55.70,44.30,37.0,3916.0,40.1,36.80,25.0,23.0,24.0
9,ICICIBANK.NS,866.0,5.52,1.10,-0.44,953.0,650.0,71.39,28.61,36.0,1094.0,24.2,20.40,26.0,10.0,18.0
51,HCLTECH.NS,1055.0,5.06,-0.50,0.09,1291.0,865.0,44.55,55.45,51.0,1117.0,16.3,20.70,6.0,22.0,14.0
43,DMART.NS,3873.0,-3.90,-0.20,-1.02,4577.0,3231.0,47.69,52.31,41.0,4282.0,138.7,109.00,11.0,18.0,14.0


In [46]:
df_prec_dev[df_prec_dev['Stock'] == 'HAVELLS.NS']

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg
38,HAVELLS.NS,1188.0,-3.38,0.16,-0.08,1392.0,1068.0,37.16,62.84,58.0,1328.0,63.7,68.4,12.0,17.0,14.0
